In [2]:
import pandas as pd

#import spark
from pyspark.sql import SparkSession
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 ass2 BNPL group 28")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

22/10/03 06:59:29 WARN Utils: Your hostname, Rudyletsgo resolves to a loopback address: 127.0.1.1; using 172.30.250.116 instead (on interface eth0)
22/10/03 06:59:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/10/03 06:59:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/10/03 06:59:31 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [46]:
## read the merchant data
merchant_info = spark.read.options(header = True).csv('../data/curated/merchant_info.csv')
merchant_tents = merchant_info.where(merchant_info.field == "tent and awning shops")
merchant_jewl = merchant_info.where(merchant_info.field == "jewelry, watch, clock, and silverware shops")
merchant_shoe = merchant_info.where(merchant_info.field == "shoe shops")

# Transform to pandas dataframe
merchant_info = merchant_info.toPandas()
merchant_tents = merchant_tents.toPandas()
merchant_jewl = merchant_jewl.toPandas()
merchant_shoe = merchant_shoe.toPandas()

## convert dataset columns into fesible types
merchant_info[['transaction_count', 'take_rate', 'total_revenue', 'mean_consumer_income', 'fraud_count', 'main_business_area_popu']] \
    = merchant_info[['transaction_count', 'take_rate', 'total_revenue', 'mean_consumer_income', 'fraud_count', 'main_business_area_popu']].apply(pd.to_numeric)

merchant_tents[['transaction_count', 'take_rate', 'total_revenue', 'mean_consumer_income', 'fraud_count', 'main_business_area_popu']] \
    = merchant_tents[['transaction_count', 'take_rate', 'total_revenue', 'mean_consumer_income', 'fraud_count', 'main_business_area_popu']].apply(pd.to_numeric)

merchant_jewl[['transaction_count', 'take_rate', 'total_revenue', 'mean_consumer_income', 'fraud_count', 'main_business_area_popu']] \
    = merchant_jewl[['transaction_count', 'take_rate', 'total_revenue', 'mean_consumer_income', 'fraud_count', 'main_business_area_popu']].apply(pd.to_numeric)

merchant_shoe[['transaction_count', 'take_rate', 'total_revenue', 'mean_consumer_income', 'fraud_count', 'main_business_area_popu']] \
    = merchant_shoe[['transaction_count', 'take_rate', 'total_revenue', 'mean_consumer_income', 'fraud_count', 'main_business_area_popu']].apply(pd.to_numeric)

#merchant_tents = merchant_info.loc[merchant_info['field'] == "tent and awning shops"].copy()
#merchant_jewl = merchant_info.loc[merchant_info['field']  == "jewelry, watch, clock, and silverware shops"].copy()
#merchant_shoe = merchant_info.loc[merchant_info['field'] == "shoe shops"].copy()

In [26]:
list(merchant_info.keys())

['merchant_abn',
 'transaction_count',
 'field',
 'take_rate',
 'revenue_level',
 'total_revenue',
 'mean_consumer_income',
 'fraud_count',
 'main_business_area_popu']

### Create score criteria(standard version)

In [27]:
## Manually set the score criteria, could be changed later.
score_criteria = { \
    'transaction_count': {'a':4, 'b': 3, 'c': 2, 'd': 1, 'e': 0}, \
        'take_rate': {'a':4, 'b': 3, 'c': 2, 'd': 1, 'e': 0}, \
            'revenue_level': {'a':0, 'b': 1, 'c': 2, 'd': 3, 'e': 4}, \
                'total_revenue': {'a':4, 'b': 3, 'c': 2, 'd': 1, 'e': 0}, \
                    'mean_consumer_income': {'a':4, 'b': 3, 'c': 2, 'd': 1, 'e': 0}, \
                        'fraud_count': {'a':0, 'b': 0, 'c': 0, 'd': 0, 'e': 0}, \
                            'main_business_area_popu': {'a':4, 'b': 3, 'c': 2, 'd': 1, 'e': 0}}

In [28]:
## store all numeric features into list for use in both Algorithms
featrue_name = list(merchant_info.keys())
featrue_name.remove('merchant_abn')
featrue_name.remove('field')

In [47]:
merchant_info

,merchant_abn,transaction_count,field,take_rate,revenue_level,total_revenue,mean_consumer_income,fraud_count,main_business_area_popu
0,16903677169,51,shoe shops,4.62,b,18506.39,795.935292,0,211425
1,21359184622,18011,motor vehicle supplies and new parts,3.60,b,5659723.28,832.634162,3,130894
2,21782713770,294,tent and awning shops,3.31,b,336040.60,796.557143,0,194701
3,23692710924,3248,"florists supplies, nursery stock, and flowers",5.61,a,969325.83,837.889286,0,116502
4,25499263184,64,"books, periodicals, and newspapers",6.14,a,32355.13,808.223438,0,44114
...,...,...,...,...,...,...,...,...,...
4021,43833568675,318,telecom,4.69,b,814552.89,791.296540,1,370220
4022,56923368251,146,"watch, clock, and jewelry repair shops",6.14,a,25387.87,768.678083,0,162711
4023,59190126409,174,"florists supplies, nursery stock, and flowers",6.87,a,60788.54,786.111494,0,198187
4024,83177825742,9206,"gift, card, novelty, and souvenir shops",5.02,b,1012564.70,833.715620,5,53266


In [29]:
score_criteria

{'transaction_count': {'a': 4, 'b': 3, 'c': 2, 'd': 1, 'e': 0},
 'take_rate': {'a': 4, 'b': 3, 'c': 2, 'd': 1, 'e': 0},
 'revenue_level': {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4},
 'total_revenue': {'a': 4, 'b': 3, 'c': 2, 'd': 1, 'e': 0},
 'mean_consumer_income': {'a': 4, 'b': 3, 'c': 2, 'd': 1, 'e': 0},
 'fraud_count': {'a': 0, 'b': 0, 'c': 0, 'd': 0, 'e': 0},
 'main_business_area_popu': {'a': 4, 'b': 3, 'c': 2, 'd': 1, 'e': 0}}

## Mark Algorithm

In [8]:
## Function used to assign continuous features into level (a,b,c,d,e) based on current merchant infomation
## merchant_info (a pd.DataFrame) is the current merchant information that are considered in our ranking system
## feature (a string) is the numeric feature we are working on
## score_criteria (a dict of dict) is the score criteria we are using for marking different features.

def mark_algorithm(merchant_info, feature, score_criteria):

    label_list = []
    score_list = []

    ## in each run of the ranking system, each numeric column is allocated with a level in (a,b,c,d,e)
    ## according to the overall quantile statistic of the feature column we are working on
    ab = merchant_info[feature].quantile(0.8)
    bc = merchant_info[feature].quantile(0.6)
    cd = merchant_info[feature].quantile(0.4)
    de = merchant_info[feature].quantile(0.2)
    
    for i in range(len(merchant_info)):
        if merchant_info.loc[i, feature] >= ab:
            label_list.append('a')
        elif (merchant_info.loc[i, feature] >= bc) & (merchant_info.loc[i, feature] < ab):
            label_list.append('b')
        elif (merchant_info.loc[i, feature] >= cd) & (merchant_info.loc[i, feature] < bc):
            label_list.append('c')
        elif (merchant_info.loc[i, feature] >= de) & (merchant_info.loc[i, feature] < cd):
            label_list.append('d')
        elif merchant_info.loc[i, feature] < de:
            label_list.append('e')

    ## mark each entry of this feature column according to the score_criteria given.
    for j in label_list:
        score_list.append(score_criteria[feature][j])

    return score_list

## Rank Algorithm

In [9]:
## Function used to rank mechant iteratively and then find the top 10 merchants to collaborate.

## merchant_info (a pd.DataFrame) is the current merchant information that are considered in our ranking system
## feature (a string) is the numeric feature we are working on
## score_criteria (a dict of dict) is the score criteria we are using for marking different features.
## remove_rate (a positive float {0,1}) is the rate to remove merchants classified as poor choice
## top_n (a positive integer) is the number of merchant to be recommended.

## Note: the lower the remove_rate, the higher the accuracy but also the longer the waiting time. (too low may cause overfitting)

def rank_algorithm(merchant_info, feature_name, score_criteria, remove_rate, top_n):
    # Stop when we find the asked number of merchants as recommendation
    while (len(merchant_info) > top_n) & (len(merchant_info) * (1-remove_rate) > top_n):
        # In each run, set the score of all merchant to zero as a fresh analysis
        merchant_info['score'] = 0

        # The main part of our rank system to mark and sum scores of all feature columns 
        for feature in feature_name:
            if feature == 'revenue_level':
                for i in range(len(merchant_info)):
                    merchant_info.loc[i, 'score'] += score_criteria[feature][merchant_info.loc[i, feature]]
            else:
                # use of mark algorthm
                merchant_info['score'] += mark_algorithm(merchant_info, feature, score_criteria)

        # sort all merchants by their total score        
        merchant_info = merchant_info.sort_values(by=['score'], ascending=False).reset_index(drop=True)
        # remove a percentage number of merchants having the lowest score (according to remove_rate given)
        merchant_info = merchant_info.drop(merchant_info.tail(int(len(merchant_info) * remove_rate)).index)
        # remove the score column after sorting and dropping merchants. (keep data integrity for next run of this algorithm)
        merchant_info = merchant_info.drop(['score'], axis=1)
        
        # if the stop criteria is not met, keep running to remove merchants that are less likely to be beneficial.
        return rank_algorithm(merchant_info, feature_name, score_criteria, remove_rate, top_n)
    
    # Show the top n merchants once stop criteria are met.
    return merchant_info.head(top_n)

In [35]:
## Run Run Run!!!!!!!!!!
top_n_merchant = rank_algorithm(merchant_info, featrue_name, score_criteria, 0.1, 100)

In [36]:
top_n_merchant

,merchant_abn,transaction_count,field,take_rate,revenue_level,total_revenue,mean_consumer_income,fraud_count,main_business_area_popu
0,40490618895,554,"gift, card, novelty, and souvenir shops",6.92,a,54264.77,948.851986,0,480087
1,32361057556,74770,"gift, card, novelty, and souvenir shops",6.61,a,8218143.78,827.606967,18,273854
2,41944909975,29998,"books, periodicals, and newspapers",0.16,e,7333635.99,823.607197,6,289380
3,89726005175,188223,tent and awning shops,6.01,a,7764006.97,824.330699,37,350738
4,79827781481,4192,"furniture, home furnishings and equipment shop...",6.82,a,8539429.10,847.735615,22,119668
5,50315283629,26006,"lawn and garden supply outlets, including nurs...",1.76,c,8381655.58,831.166096,9,270898
6,48534649627,57872,"opticians, optical goods, and eyeglasses",6.64,a,8200624.34,834.195233,14,172424
7,45629217853,198936,"gift, card, novelty, and souvenir shops",6.98,a,7325106.13,824.636026,43,261909
8,73761689784,859,motor vehicle supplies and new parts,4.17,b,1262563.04,938.309546,0,497071
9,27093785141,22715,"stationery, office supplies and printing and w...",2.73,c,8567686.31,835.831283,5,232352


## Prepare the segment merchant data for ranking
### Three selected segments are "tent and awning shops", "telecom" and "shoe shops"

## Design a score criteria with high priority on take rate
### specifically for the segment jewlery

### Buiness rule
No.1: Merchant care more about the consumption ability of their target customers (High mean income & low population > low income & large population )

No.2: The BNPL company tends to select the merchants allow them to earn more (High take rate is prefered)

No.3: Combined with No.2, the BNPL company like the merchant has frequent transactions (Large transactions count is prefered)

No.3 alternative: BNPL company may like the merchant has large total revenue 

No.4: Large penalty on fraud probability as this segment is related to luxury goods

In [50]:
top_10_merchant_jewlery = rank_algorithm(merchant_jewl, featrue_name, score_criteria, 0.2, 10)

In [51]:
# This is the top 10 jewlery shops when using standard criteria
top_10_merchant_jewlery

,merchant_abn,transaction_count,field,take_rate,revenue_level,total_revenue,mean_consumer_income,fraud_count,main_business_area_popu
0,19492220327,723,"jewelry, watch, clock, and silverware shops",4.93,b,7198756.77,815.730705,128,232303
1,15043504837,175,"jewelry, watch, clock, and silverware shops",4.62,b,2912189.09,801.674285,62,237617
2,90918180829,493,"jewelry, watch, clock, and silverware shops",2.62,c,5072774.57,787.019473,100,242001
3,75265429612,16,"jewelry, watch, clock, and silverware shops",4.92,b,156113.21,1942.700001,4,624063
4,23709946765,170,"jewelry, watch, clock, and silverware shops",3.59,b,1270977.51,789.755883,20,262181
5,12983555508,6,"jewelry, watch, clock, and silverware shops",5.09,b,55507.02,813.250000,1,478481
6,41331227038,34,"jewelry, watch, clock, and silverware shops",3.97,b,157573.05,810.338237,0,618091
7,26793253394,12,"jewelry, watch, clock, and silverware shops",2.36,c,196846.83,832.208328,2,396777
8,66228393506,92,"jewelry, watch, clock, and silverware shops",1.92,c,1118797.54,993.461957,17,77875
9,86889657711,66,"jewelry, watch, clock, and silverware shops",4.65,b,1079196.73,816.234851,24,115036


### Next want to see the rank when using designed score crietria

In [61]:
score_criteria_jewlery = { \
    'transaction_count': {'a':8, 'b': 6, 'c': 4, 'd': 2, 'e': 0}, \
        'take_rate': {'a':8, 'b': 6, 'c': 4, 'd': 2, 'e': 0}, \
            'revenue_level': {'a':0, 'b': 1, 'c': 2, 'd': 3, 'e': 4}, \
                'total_revenue': {'a': 6, 'b': 4.5, 'c': 3, 'd': 1.5, 'e': 0}, \
                    'mean_consumer_income': {'a':8, 'b': 6, 'c': 4, 'd': 2, 'e': 0}, \
                        'fraud_count': {'a':0, 'b': 0, 'c': 0, 'd': 0, 'e': 0}, \
                            'main_business_area_popu': {'a':4, 'b': 3.5, 'c': 3, 'd': 2.5, 'e': 2}}

In [62]:
top_10_merchant_jewlery_designed = rank_algorithm(merchant_jewl, featrue_name, score_criteria_jewlery, 0.2, 10)

In [63]:
top_10_merchant_jewlery_designed

,merchant_abn,transaction_count,field,take_rate,revenue_level,total_revenue,mean_consumer_income,fraud_count,main_business_area_popu
0,19492220327,723,"jewelry, watch, clock, and silverware shops",4.93,b,7198756.77,815.730705,128,232303
1,15043504837,175,"jewelry, watch, clock, and silverware shops",4.62,b,2912189.09,801.674285,62,237617
2,90918180829,493,"jewelry, watch, clock, and silverware shops",2.62,c,5072774.57,787.019473,100,242001
3,23709946765,170,"jewelry, watch, clock, and silverware shops",3.59,b,1270977.51,789.755883,20,262181
4,75265429612,16,"jewelry, watch, clock, and silverware shops",4.92,b,156113.21,1942.700001,4,624063
5,66228393506,92,"jewelry, watch, clock, and silverware shops",1.92,c,1118797.54,993.461957,17,77875
6,44345785419,81,"jewelry, watch, clock, and silverware shops",6.07,a,1211300.90,791.981481,23,164784
7,10596295795,97,"jewelry, watch, clock, and silverware shops",6.84,a,1122230.91,765.761857,19,146029
8,29253286472,173,"jewelry, watch, clock, and silverware shops",3.53,b,1326580.35,783.850869,17,201176
9,57860746842,99,"jewelry, watch, clock, and silverware shops",4.64,b,1115039.24,806.882828,16,40837


### specifically for the segment shoe shops

No.1: Merchant care more about the numbers of potential customers (low mean income & large population > High income & small population )

No.2: The BNPL company tends to select the merchants considered to be stable (large total transactions count is prefered)

No.3 small penalty on fraud probability as shoes are usually considered to be affordable

### specifically for the segment tent shops

No.1: The BNPL company tends to select the merchants considered to be stable (large total transactions count and large total revenue is prefered)

No.2 large penalty on fraud probability as clients dont like the risks

In [14]:
# Higher priority means the algorithm tends to select the merchant that can make the BNPL company earn more on each transaction
